# Gradient-free `CRP` parameter optimization

In [1]:
import pennylane as qml
from pennylane import numpy as np
from numpy.typing import NDArray

import math

from scipy.optimize import minimize

from matplotlib import pyplot as plt

## 1. Set up demo circuit

In [2]:
dev = qml.device("default.qubit", wires=4, shots=None)

num_layers = 5
num_qubits = 4

@qml.qnode(dev)
def an_interesting_circuit(params: NDArray[np.float_]):
    for layer in range(num_layers):
        for qubit in range(num_qubits):
            qml.RX(params[layer, qubit, 0], wires=qubit)
            qml.RZ(params[layer, qubit, 1], wires=qubit)
        for qubit in range(num_qubits - 2, -1, -1): # incl. start, excl. stop, step
            qml.CRZ(params[layer, qubit, 2], (qubit + 1, qubit))
    return qml.expval(qml.PauliZ(1))

print(qml.draw(an_interesting_circuit)(np.random.random((num_layers, num_qubits, 3))))

0: ──RX(0.20)──RZ(0.02)─────────────────────╭RZ(0.19)──RX(0.47)──RZ(0.76)───────────╭RZ(0.81)
1: ──RX(0.04)──RZ(0.24)───────────╭RZ(0.37)─╰●─────────RX(0.26)──RZ(0.76)─╭RZ(0.48)─╰●───────
2: ──RX(0.41)──RZ(0.19)─╭RZ(0.71)─╰●─────────RX(1.00)──RZ(0.13)─╭RZ(0.02)─╰●─────────RX(0.06)
3: ──RX(0.71)──RZ(0.53)─╰●─────────RX(0.60)──RZ(0.10)───────────╰●─────────RX(0.31)──RZ(0.04)

───RX(0.56)──RZ(0.82)───────────╭RZ(0.18)──RX(0.11)──RZ(0.15)───────────╭RZ(0.96)──RX(0.11)
───RX(0.37)──RZ(0.26)─╭RZ(0.18)─╰●─────────RX(0.89)──RZ(0.68)─╭RZ(0.79)─╰●─────────RX(0.24)
───RZ(0.16)─╭RZ(0.37)─╰●─────────RX(0.09)──RZ(0.53)─╭RZ(0.05)─╰●─────────RX(0.40)──RZ(0.18)
────────────╰●─────────RX(0.23)──RZ(0.37)───────────╰●─────────RX(0.85)──RZ(0.03)──────────

───RZ(0.71)───────────╭RZ(0.95)─┤     
───RZ(0.45)─╭RZ(0.69)─╰●────────┤  <Z>
──╭RZ(0.52)─╰●──────────────────┤     
──╰●────────────────────────────┤     


## 2. Optimization loop

In [3]:
def create_univariate(circuit, params: NDArray[np.int_], param_index):
    def univariate(param_value):
        updated_params = params.copy()
        updated_params[param_index] = param_value
        return circuit(updated_params)
    
    return univariate

In [4]:
from reconstruction import reconstruct
from minimization import minimize_reconstruction

# in each iteration, reconstruct and optimize the univariate cost functions independently
def crotosolve_iteration(circuit, params, debug = False):
    final_value = 42

    iterator = np.nditer(params, flags=['multi_index'])
    for old_param_value in iterator:
        param_index = iterator.multi_index
        if debug: print(f"Optimizing parameter {param_index}...")
        univariate = create_univariate(circuit, params, param_index)
        reconstruction = reconstruct(univariate)
        new_param_value, new_fun_value = minimize_reconstruction(reconstruction)
        if debug: print(f"Parameter update for {param_index} from {old_param_value} to {new_param_value} ({new_fun_value})")
        params[param_index] = new_param_value
        final_value = new_fun_value
    
    return final_value

In [5]:
def crotosolve(circuit, initial_params, debug = False):
    dataset = []

    params = initial_params.copy()
    for iteration in range(5):
        if debug: print(f" ===== ITERATION NO {iteration} =====")
        new_fun_value = crotosolve_iteration(circuit, params, debug=debug)
        dataset.append(new_fun_value)
    
    if debug: print(dataset)
    return params

rng = np.random.default_rng()
params = rng.random((5, 4, 3))

crotosolve(an_interesting_circuit, params, debug=True)

 ===== ITERATION NO 0 =====
Optimizing parameter (0, 0, 0)...
Parameter update for (0, 0, 0) from 0.19378533587461022 to 5.941728973542421 (-0.3014141692681688)
Optimizing parameter (0, 0, 1)...
Parameter update for (0, 0, 1) from 0.09948335959035792 to 5.9822997693257705 (-0.30476429404718797)
Optimizing parameter (0, 0, 2)...
Parameter update for (0, 0, 2) from 0.567107038601407 to 2.6277090896481035 (-0.5691065719246786)
Optimizing parameter (0, 1, 0)...
Parameter update for (0, 1, 0) from 0.7788359603874218 to 1.533805818202023 (-0.7776944166727441)
Optimizing parameter (0, 1, 1)...
Parameter update for (0, 1, 1) from 0.5584126255085469 to 0.5180360978713958 (-0.7783197571535219)
Optimizing parameter (0, 1, 2)...
Parameter update for (0, 1, 2) from 0.887793521931696 to 8.962200062069993 (-0.8862702502282414)
Optimizing parameter (0, 2, 0)...
Parameter update for (0, 2, 0) from 0.15441890948851278 to 0.5112602171099072 (-0.9431497492564438)
Optimizing parameter (0, 2, 1)...
Paramete

tensor([[[5.96517072, 6.00756611, 2.62812314],
         [1.5202235 , 0.51846993, 8.90345339],
         [0.50363106, 0.34502104, 6.28211631],
         [0.10216312, 0.45467191, 3.14159265]],

        [[0.70326008, 5.11728321, 0.33942501],
         [0.26632303, 0.38904569, 0.86812707],
         [0.99531205, 0.94670343, 6.31618507],
         [0.21928255, 0.13345633, 3.14159265]],

        [[0.82815308, 1.77319071, 0.74474528],
         [0.70242478, 0.50164108, 0.48098806],
         [0.09026518, 4.88258894, 0.11457878],
         [3.24693878, 3.14159265, 3.14159265]],

        [[5.25648717, 3.29867229, 2.33668178],
         [0.84512275, 0.77442121, 2.18471941],
         [0.14536505, 3.29867229, 3.14159265],
         [3.14159265, 2.98451302, 3.14159265]],

        [[3.14159265, 3.29867229, 3.29867229],
         [6.33253934, 2.98451302, 3.14159265],
         [3.14159265, 3.14159265, 3.14159265],
         [3.14159265, 3.14159265, 3.14159265]]], requires_grad=True)